동일한 단백질의 conformation이 다른 경우, RMSF를 계산한다. res_num, res_type을 비교해서 merge하고(두 값이 다른 경우에는 빠짐), RMSD를 구한다.

In [235]:
import pandas as pd
import math
import os 

In [236]:
path = '/home/siu/temp/BAB-301_st'
os.chdir(path)

## Parse PDB. Extract Atom Names & Coords

In [239]:
fileA = 'ph75.pdb'
fileB = '200.pdb'

In [240]:
# parse pdb file

f = open(fileA, mode='r')
tableA = pd.DataFrame()

for line in f:
    if line.startswith('ATOM'):
        row = line.split()[:11]
        tableA = pd.concat([tableA, pd.DataFrame([row])], ignore_index=True)

tableA[:10]

,0,1,2,3,4,5,6,7,8,9,10
0,ATOM,1,N,GLY,A,29,138.224,144.116,143.062,1.00,74.75
1,ATOM,2,CA,GLY,A,29,136.861,144.587,143.223,1.00,71.34
2,ATOM,3,C,GLY,A,29,136.780,145.980,143.816,1.00,71.17
3,ATOM,4,O,GLY,A,29,137.746,146.469,144.401,1.00,73.15
4,ATOM,5,N,ILE,A,30,135.622,146.619,143.654,1.00,58.70
5,ATOM,6,CA,ILE,A,30,135.374,147.953,144.193,1.00,57.40
6,ATOM,7,C,ILE,A,30,135.564,147.933,145.706,1.00,56.60
7,ATOM,8,O,ILE,A,30,134.959,147.115,146.407,1.00,60.35
8,ATOM,9,CB,ILE,A,30,133.966,148.457,143.816,1.00,58.09
9,ATOM,10,CG1,ILE,A,30,133.816,148.588,142.296,1.00,57.32


In [241]:
# parse pdb file
f = open(fileB, mode='r')
tableB = pd.DataFrame()

for line in f:
    if line.startswith('ATOM'):
        row = line.split()[:11]
        tableB = pd.concat([tableB, pd.DataFrame([row])], ignore_index=True)

tableB[:10]

,0,1,2,3,4,5,6,7,8,9,10
0,ATOM,1,N,GLY,A,29,138.720,144.273,144.390,1.00,80.44
1,ATOM,2,CA,GLY,A,29,137.366,144.855,144.366,1.00,81.22
2,ATOM,3,C,GLY,A,29,137.423,146.357,144.560,1.00,82.82
3,ATOM,4,O,GLY,A,29,138.525,146.875,144.805,1.00,81.50
4,ATOM,5,N,ILE,A,30,136.284,147.037,144.448,1.00,74.40
5,ATOM,6,CA,ILE,A,30,136.289,148.523,144.543,1.00,70.31
6,ATOM,7,C,ILE,A,30,136.337,148.904,146.024,1.00,71.34
7,ATOM,8,O,ILE,A,30,135.297,149.330,146.553,1.00,75.17
8,ATOM,9,CB,ILE,A,30,135.066,149.116,143.817,1.00,72.32
9,ATOM,10,CG1,ILE,A,30,134.758,148.376,142.513,1.00,72.21


## Calculate RMSF in DataFrame

In [242]:
df = pd.merge(tableA, tableB, on=[2,3,4,5], how='inner', suffixes=('_A', '_B'))

In [243]:
# calc rmsf
df = df.astype({'6_A' : 'float', '7_A' : 'float', '8_A' : 'float', '6_B' : 'float', '7_B' : 'float', '8_B' : 'float'})
df['rmsf'] = ( ( (df['6_A'] - df['6_B'])**2 + (df['7_A'] - df['7_B'])**2 + (df['8_A'] - df['8_B'])**2 ) )
df['rmsf'] = df['rmsf'].apply(lambda x: math.sqrt(x))

# arrange columns
df = df[[2, 3,4,5, 'rmsf']]
df.columns = ['atom', 'residue', 'chain', 'resnum', 'rmsf']
df

,atom,residue,chain,resnum,rmsf
0,N,GLY,A,29,1.426271
1,CA,GLY,A,29,1.278005
2,C,GLY,A,29,1.053145
3,O,GLY,A,29,0.966899
4,N,ILE,A,30,1.115080
...,...,...,...,...,...
6177,N,ALA,B,471,1.789703
6178,CA,ALA,B,471,1.539813
6179,C,ALA,B,471,1.550328
6180,O,ALA,B,471,1.581367


In [245]:
# ca
ca = df.loc[df['atom'] == 'CA', :].groupby(['residue', 'chain', 'resnum'], sort=False).mean(numeric_only=True).reset_index()
ca.rename(columns={'rmsf': 'ca'}, inplace=True)

# all
all = df.groupby(['residue', 'chain', 'resnum'], sort=False).mean(numeric_only=True).reset_index()
all.rename(columns={'rmsf': 'all'}, inplace=True)

# side
resi = df[~df['atom'].isin(['N', 'CA', 'C', 'O'])].groupby(['residue', 'chain', 'resnum'], sort=False).mean(numeric_only=True).reset_index()
resi.rename(columns={'rmsf': 'side'}, inplace=True)

# To DataFrame
rmsf = all.merge(resi, on=['residue', 'chain', 'resnum']).merge(ca, on=['residue', 'chain', 'resnum'])
rmsf['all'] = rmsf['all'].round(2)
rmsf['side'] = rmsf['side'].round(2)
rmsf['ca'] = rmsf['ca'].round(2)
rmsf

,residue,chain,resnum,all,side,ca
0,ILE,A,30,1.41,1.37,1.13
1,GLN,A,31,1.20,1.49,1.07
2,CYS,A,32,1.37,1.61,1.10
3,SER,A,33,1.45,1.50,1.39
4,GLN,A,34,1.19,1.68,0.50
...,...,...,...,...,...,...
755,VAL,B,466,1.68,1.61,1.71
756,LEU,B,468,2.66,2.94,2.18
757,ALA,B,469,2.59,2.71,2.70
758,LEU,B,470,2.79,3.68,2.09


In [246]:
rmsf.to_csv('rmsf.csv', index=False)

In [247]:
rmsf_A = rmsf[rmsf['chain'] == 'A']

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=rmsf_A['resnum'], y=rmsf_A['all'], name='All'))
fig.add_trace(go.Scatter(x=rmsf_A['resnum'], y=rmsf_A['side'], name='Side Chain'))
fig.add_trace(go.Scatter(x=rmsf_A['resnum'], y=rmsf_A['ca'], name='Ca'))
fig.update_layout(title='RMSF', xaxis_title='Residue Number', yaxis_title='RMSF (Å)', width=1200, height=800)
fig.show()

In [248]:
# copy and paste printed text to maestro commands input
chain = 'A'
rmsf_criteria = 2
res_list = rmsf.loc[(rmsf['chain'] == chain) & (rmsf['side'] > rmsf_criteria), :]['resnum'].tolist()
print(f"workspaceselectionreplace (chain.name {chain}) AND (res.num {str(res_list).replace('[','').replace(']','')})")

workspaceselectionreplace (chain.name A) AND (res.num '35', '56', '59', '60', '67', '68', '72', '104', '135', '166', '169', '171', '174', '177', '178', '182', '183', '186', '204', '207', '209', '255', '256', '259', '279', '282', '283', '289', '291', '292', '293', '294', '295', '296', '298', '339', '361', '371', '372', '373', '374', '375', '406', '420', '452', '465', '468', '469', '470')
